<a href="https://colab.research.google.com/github/dusihan-hub/Curso2025-2026-DataScience/blob/Assignment4/Task07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib
import urllib.request
url = 'https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2025-2026/refs/heads/master/Assignment4/course_materials/python/validation.py'
urllib.request.urlretrieve(url, 'validation.py')
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2025-2026/master/Assignment4/course_materials"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.4/565.4 kB 11.8 MB/s eta 0:00:00


In [2]:
from validation import Report

First let's read the RDF file

In [3]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
# Do not change the name of the variables
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.parse(github_storage+"/rdf/data06.ttl", format="TTL")
report = Report()

**TASK 7.1a: For all classes, list each classURI. If the class belogs to another class, then list its superclass.**
**Do the exercise in RDFLib returning a list of Tuples: (class, superclass) called "result". If a class does not have a super class, then return None as the superclass**

In [6]:
# TO DO
classes = set()

for c in g.subjects(RDF.type, RDFS.Class):
    classes.add(c)

for c in g.objects(None, RDF.type):
    classes.add(c)

result = []
for c in classes:
    superclass = None
    for s in g.objects(c, RDFS.subClassOf):
        superclass = s
        break
    result.append((c, superclass))

# Visualize the results
for r in result:
  print(r)

(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#InterimAssociateProfessor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#AssociateProfessor'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#FullProfessor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Professor'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Student'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Person'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Professor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Person'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#AssociateProfessor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Professor'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Person'), None)
(rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Class'), None)
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Animal'), None)


In [7]:
## Validation: Do not remove
report.validate_07_1a(result)

ERROR: The number of classes returned is not correct
The class http://www.w3.org/2000/01/rdf-schema#Class has no superclass
ERROR: incorrect class retrieved


**TASK 7.1b: Repeat the same exercise in SPARQL, returning the variables ?c (class) and ?sc (superclass)**

In [8]:
query = """
SELECT ?c ?sc
WHERE {
    ?c a rdfs:Class .
    OPTIONAL { ?c rdfs:subClassOf ?sc . }
    FILTER(STRSTARTS(STR(?c), "http://oeg.fi.upm.es/def/people#"))
}
"""

for r in g.query(query):
  print(r.c, r.sc)


http://oeg.fi.upm.es/def/people#Person None
http://oeg.fi.upm.es/def/people#Animal None
http://oeg.fi.upm.es/def/people#Professor http://oeg.fi.upm.es/def/people#Person
http://oeg.fi.upm.es/def/people#Student http://oeg.fi.upm.es/def/people#Person
http://oeg.fi.upm.es/def/people#FullProfessor http://oeg.fi.upm.es/def/people#Professor
http://oeg.fi.upm.es/def/people#AssociateProfessor http://oeg.fi.upm.es/def/people#Professor
http://oeg.fi.upm.es/def/people#InterimAssociateProfessor http://oeg.fi.upm.es/def/people#AssociateProfessor


In [9]:
## Validation: Do not remove
report.validate_07_1b(query,g)

TASK 7.1b OK


**TASK 7.2a: List all individuals of "Person" with RDFLib (remember the subClasses). Return the individual URIs in a list called "individuals"**


In [10]:
ns = Namespace("http://oeg.fi.upm.es/def/people#")
# variable to return
individuals = []
classes = set()
classes.add(ns.Person)
def get_subclasses(c):
    for s in g.subjects(RDFS.subClassOf, c):
        if s not in classes:
            classes.add(s)
            get_subclasses(s)

get_subclasses(ns.Person)

for c in classes:
    for ind in g.subjects(RDF.type, c):
        individuals.append(ind)

# visualize results
for i in individuals:
  print(i)

http://oeg.fi.upm.es/def/people#Raul
http://oeg.fi.upm.es/def/people#Asun
http://oeg.fi.upm.es/def/people#Oscar


In [11]:
# validation. Do not remove
report.validate_07_02a(individuals)

TASK 7.2a OK


**TASK 7.2b: Repeat the same exercise in SPARQL, returning the individual URIs in a variable ?ind**

In [12]:
query = """
SELECT ?ind
WHERE {
    ?ind a ?c .
    ?c rdfs:subClassOf* <http://oeg.fi.upm.es/def/people#Person> .
}
"""


for r in g.query(query):
  print(r.ind)
# Visualize the results

http://oeg.fi.upm.es/def/people#Asun
http://oeg.fi.upm.es/def/people#Oscar
http://oeg.fi.upm.es/def/people#Raul


In [13]:
## Validation: Do not remove
report.validate_07_02b(g, query)

TASK 7.2b OK


**TASK 7.3:  List the name and type of those who know Rocky (in SPARQL only). Use name and type as variables in the query**

In [19]:
query = """
SELECT ?name ?type
WHERE {
    ?person ns:knows ns:Rocky .
    ?person a ?type .
    OPTIONAL { ?person foaf:name ?name }
    FILTER(STRSTARTS(STR(?type), "http://oeg.fi.upm.es/def/people#"))
}
"""

# TO DO
# Visualize the results
for r in g.query(query):
  print(r.name, r.type)


In [20]:
## Validation: Do not remove
report.validate_07_03(g, query)

ERROR: The number of individuals returned is not correct


**Task 7.4: List the name of those entities who have a colleague with a dog, or that have a collegue who has a colleague who has a dog (in SPARQL). Return the results in a variable called name**

In [23]:
query = """

SELECT DISTINCT ?name
WHERE {
    ?person ns:colleague ?col1 .

    {
        ?col1 ns:hasPet ?pet1 .
        ?pet1 a ns:Dog .
    }

    UNION
    {
        ?col1 ns:colleague ?col2 .
        ?col2 ns:hasPet ?pet2 .
        ?pet2 a ns:Dog .
    }

    ?person ns:name ?name .
}
"""
for r in g.query(query):
  print(r.name)

# TO DO
# Visualize the results

In [24]:
## Validation: Do not remove
report.validate_07_04(g,query)
report.save_report("_Task_07")

ERROR: The number of individuals returned is not correct
